In [1]:
import pandas as pd
import numpy as np
import os
import acquire_nick as get
import prepare_nick as prep
import re
import wrangle as w

In [11]:
df = get.acquire_data_new()

Using cached CSV


In [2]:
df = w.wrangle_df()

Using cached CSV


KeyError: "None of ['id'] are in the columns"

In [3]:
df.shape

(7282, 19)

In [14]:
df.columns

Index(['release_date', 'budget', 'genres', 'id', 'imdb_id', 'overview',
       'popularity', 'production_companies', 'production_countries', 'revenue',
       'runtime', 'title', 'vote_average', 'vote_count', 'cast_actor_1',
       'cast_actor_2', 'cast_actor_3', 'total_n_cast', 'release_year',
       'profitable'],
      dtype='object')

In [12]:
df = df.reset_index()

In [15]:
df = df[['title', 'genres', 'cast_actor_1',
       'cast_actor_2', 'cast_actor_3', 'total_n_cast',
         'budget', 'revenue', 'vote_average', 'vote_count',
         'production_companies', 'production_countries',
         'overview', 'popularity', 'runtime', 'profitable',
         'release_date', 'release_year', 'runtime']]

In [16]:
df

,title,genres,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,budget,revenue,vote_average,vote_count,production_companies,production_countries,overview,popularity,runtime,profitable,release_date,release_year,runtime
0,The Birth of a Nation,Drama History War,Lillian Gish,Mae Marsh,Henry B. Walthall,49.0,8000000.0,11000000.0,6.4,109.0,EpochFilmCo.,UnitedStatesofAmerica,"The Birth of A Nation is a silent film from 1915 and the highest grossing silent film in film history. The film tells a romance story during the American civil war. D.W. Griffith invested heavily in its high production values, pioneering many new camera effects. The Birth of a Nation was strongly protested for its negative portrayal of newly freed slaves (mostly white actors in blackface), which went on to create and propagate negative images of blacks of the early 20th century in the United States. The film was used as a recruiting propaganda by the KKK until the 1940s.",5.113205,193.0,False,1915-02-08,1915,193.0
1,The Cheat,Drama,Fannie Ward,Jack Dean,Sessue Hayakawa,11.0,8000000.0,137365.0,4.8,11.0,JesseL.LaskyFeaturePlayCompany,UnitedStatesofAmerica,"A venal, spoiled stockbroker's wife impulsively embezzles $10,000 from the charity she chairs and desperately turns to a Burmese ivory trader to replace the stolen money.",0.680407,59.0,False,1915-12-13,1915,59.0
2,"20,000 Leagues Under the Sea",Adventure Drama Action Science Fiction,Lois Alexander,Curtis Benton,Wallace Clarke,13.0,8000000.0,8000000.0,6.0,7.0,TheUniversalFilmMfg.Co.,Bahamas UnitedStatesofAmerica,"Captain Nemo has built a fantastic submarine for his mission of revenge. He has traveled over 20,000 leagues in search of Charles Denver - a man who caused the death of Princess Daaker. Seeing what he had done, Denver took the daughter to his yacht and sailed away.",0.917777,105.0,False,1916-12-24,1916,105.0
3,Mickey,Comedy Drama,Mabel Normand,George Nichols,Wheeler Oakman,8.0,8000000.0,8000000.0,7.5,4.0,MabelNormandFeatureFilmCompany,UnitedStatesofAmerica,"Mickey, an orphan who has been brought up in a mining settlement, is sent to New York to live with her aunt.",0.211008,93.0,False,1918-08-01,1918,93.0
4,The Kid,Comedy Drama,Charlie Chaplin,Jackie Coogan,Edna Purviance,35.0,8000000.0,2500000.0,8.0,404.0,CharlesChaplinProductions FirstNationalPictures,UnitedStatesofAmerica,"Considered one of Charlie Chaplin's best films, The Kid also made a star of little Jackie Coogan, who plays a boy cared for by The Tramp when he's abandoned by his mother, Edna. Later, Edna has a change of heart and aches to be reunited with her son. When she finds him and wrests him from The Tramp, it makes for what turns out be one of the most heart-wrenching scenes ever included in a comedy.",8.168456,68.0,False,1921-01-21,1921,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7277,Wind River,Action Crime Mystery Thriller,Jeremy Renner,Elizabeth Olsen,Gil Birmingham,22.0,11000000.0,184770205.0,7.4,181.0,ThunderRoadPictures StarThrowerEntertainment AcaciaFilmedEntertainment,UnitedKingdom UnitedStatesofAmerica Canada,An FBI agent teams with the town's veteran game tracker to investigate a murder that occurred on a Native American reservation.,40.796775,111.0,True,2017-08-03,2017,111.0
7278,The Dark Tower,Action Western Science Fiction Fantasy Horror,Idris Elba,Matthew McConaughey,Tom Taylor,50.0,60000000.0,71000000.0,5.7,688.0,ImagineEntertainment WeedRoadPictures MediaRightsCapital SonyPicturesEntertainment,SouthAfrica UnitedStatesofAmerica,"The last Gunslinger, Roland Deschain, has been locked in an eternal battle with Walter O’Dim, also known as the Man in Black, determined to prevent him from toppling the Dark Tower, which holds the universe together. With the fate of the worlds at stake, good and evil will collide in the ultimate battle as only Roland can defend the Tower from the Man in Black.",50.903593,95.0,False,2017-08-03,2017,95.0
7279,Kidnap,Drama Thriller,Halle Berry,Sage Correa,C

In [7]:
import mitosheet
mitosheet.sheet(df, analysis_to_replay="id-cyawlxowbb")

MitoWidget(analysis_data_json='{"analysisName": "id-cyawlxowbb", "analysisToReplay": null, "code": [], "stepSu…

In [9]:
df.id.is_unique

True

In [ ]:
from mitosheet import *; # Analysis Name:id-cyawlxowbb;
    
# Reordered column title
df_columns = [col for col in df.columns if col != 'title']
df_columns.insert(0, 'title')
df = df[df_columns]
